# Cross Validation in `Python`

One of the main goals of the statistical and machine learning (ML) models is to predict a response variable (in many cases this is the unique goal).

In this article we are going to study different algorithms oriented to measure the predictive power of a statistic or ML model.

## Train data-set

## Test data-set

## Bibliography

https://www.w3schools.com/python/python_ml_cross_validation.asp

# K=10 FOLDS CV


In [1]:
import pandas as pd
import numpy as np


In [2]:

url = 'https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/Boston.csv'

Boston = pd.read_csv(url)

In [15]:
url = 'https://raw.githubusercontent.com/FabioScielzoOrtiz/Estadistica4all-blog/main/Linear%20Regression%20in%20Python%20and%20R/properties_data.csv'

House_Price = pd.read_csv(url)

In [4]:
def varcharProcessing(X, varchar_process = "dummy_dropfirst"):
    
    dtypes = X.dtypes

    if varchar_process == "drop":   
        X = X.drop(columns = dtypes[dtypes == np.object].index.tolist())

    elif varchar_process == "dummy":
        X = pd.get_dummies(X,drop_first=False)

    elif varchar_process == "dummy_dropfirst":
        X = pd.get_dummies(X,drop_first=True)

    else: 
        X = pd.get_dummies(X,drop_first=True)
    
    X["intercept"] = 1
    cols = X.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    X = X[cols]
    
    return X

In [19]:
House_Price = House_Price.loc[: , (House_Price.columns != 'neighborhood') &  (House_Price.columns != 'id') ]

In [24]:
House_Price = varcharProcessing(House_Price, varchar_process = "dummy_dropfirst")

In [25]:
House_Price.dtypes

intercept                int64
latitude               float64
longitude              float64
price                    int64
size_in_sqft             int64
price_per_sqft         float64
no_of_bedrooms           int64
no_of_bathrooms          int64
maid_room                 bool
unfurnished               bool
balcony                   bool
barbecue_area             bool
built_in_wardrobes        bool
central_ac                bool
childrens_play_area       bool
childrens_pool            bool
concierge                 bool
covered_parking           bool
kitchen_appliances        bool
lobby_in_building         bool
maid_service              bool
networked                 bool
pets_allowed              bool
private_garden            bool
private_gym               bool
private_jacuzzi           bool
private_pool              bool
security                  bool
shared_gym                bool
shared_pool               bool
shared_spa                bool
study                     bool
vastu_co

In [27]:
model = LinearRegression().fit(House_Price.loc[: , House_Price.columns != 'price' ], House_Price.loc[: , House_Price.columns == 'price' ])

In [49]:
1/6

0.16666666666666666

In [30]:
from sklearn.linear_model import LinearRegression

In [71]:
def Repeted_K_10_Fold_CV( Data, response_name , K , n_iter ):


    from sklearn.utils import resample

    ECM_Repeted_K_Folds_vector = []


    for iter in range(0, n_iter):

        sample = resample(range(0, len(Data)), n_samples=len(Data), replace=False, random_state=iter)

        df_sample = pd.DataFrame({'index': range(0,len(Data)) , 'sample':sample})

        
        limites = []

        
        for q in np.arange(0 , 1 + 1/K , 1/K):

            limites.append( np.quantile( range(0, len(Data)) , q ).round(0) )



        ECM_K_FOLDS_vector = []

        for j in range(1, len(limites)-1):

            X_test = Data.loc[df_sample.loc[limites[j]:limites[j+1], 'sample'] , Data.columns != response_name ] 
            Y_test = Data.loc[df_sample.loc[limites[j]:limites[j+1], 'sample'] , Data.columns == response_name ]

            X_train = Data.loc[ : , Data.columns != response_name ].drop(df_sample.loc[limites[j]:limites[j+1], 'sample'] )
            Y_train = Data.loc[ : ,  Data.columns == response_name ].drop(df_sample.loc[limites[j]:limites[j+1], 'sample'])

            Y_test = Y_test.to_numpy()

            model = LinearRegression().fit(X_train, Y_train)

            ECM_K_FOLDS_vector.append( (( model.predict(X_test) - Y_test )**2 ).sum() / len(Y_test) )

    
        ECM_Repeted_K_Folds_vector.append( np.array(ECM_K_FOLDS_vector).mean() )


    
    ECM_Repeted_K_Folds = np.array( ECM_Repeted_K_Folds_vector ).mean()
    
    return(ECM_Repeted_K_Folds_vector , ECM_Repeted_K_Folds)

In [75]:
ECM_Repeted_K_Folds_vector , ECM_Repeted_K_Folds = Repeted_K_10_Fold_CV( Data=House_Price , response_name='price' , K=10 , n_iter=50 )

In [76]:
ECM_Repeted_K_Folds

867229286465.9777